In [4]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')

# -------------------------------
# PARAMETERS
# -------------------------------
aez = 10                                                # AEZ number
num_years = 7                                           # Number of assets produced (adjust)
start_year = 2017                                       # First year available
base = f"projects/raman-461708/assets/AEZ_{aez}_v4_temporal_3years__"


# -------------------------------
# LOAD ALL ASSETS FOR GIVEN AEZ
# -------------------------------
images = []

for i in range(num_years):
    y1 = start_year + i
    y2 = y1 + 1
    
    asset = f"{base}{y1}-07-01_{y2}-06-30"
    try:
        img = ee.Image(asset)
        images.append(img)
        print(f"Loaded → {asset}")
    except:
        print(f"⚠ Missing or inaccessible → {asset}")
        
L1_asset_new = images

Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2017-07-01_2018-06-30
Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2018-07-01_2019-06-30
Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2019-07-01_2020-06-30
Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2020-07-01_2021-06-30
Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2021-07-01_2022-06-30
Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2022-07-01_2023-06-30
Loaded → projects/raman-461708/assets/AEZ_10_v4_temporal_3years__2023-07-01_2024-06-30


In [ ]:
#temporal correction for background pixels


length = len(L1_asset_new)

# intermediate years
for i in range(1, length-1):

    before = L1_asset_new[i - 1]
    middle = L1_asset_new[i]
    after = L1_asset_new[i + 1]

    # any-bg-any
    cond1 = (before.gte(1)) \
        .And(after.gte(1)) \
        .And(middle.eq(0))

    if i == 1:
        middle = middle.where(cond1, after)
    elif i==3 :
        middle = middle.where(cond1, before)
    else:
        middle = middle.where(cond1, before)

    L1_asset_new[i] = middle

# first year
cond1 = L1_asset_new[0].eq(0) \
        .And(L1_asset_new[1].gte(1))
L1_asset_new[0] = L1_asset_new[0].where(cond1, L1_asset_new[1])

# last year
cond2 = L1_asset_new[length-1].eq(0) \
        .And(L1_asset_new[length-2].gte(1))
L1_asset_new[length-1] = L1_asset_new[length-1].where(cond2, L1_asset_new[length-2])

# temporall correction for intermediate years

zero_image = ee.Image.constant(0).clip(L1_asset_new[0].geometry())

# Calculating condition count for each pixel
for i in range(1, length-1):
    before = L1_asset_new[i - 1]
    middle = L1_asset_new[i]
    after = L1_asset_new[i + 1]

    # shrubs-green-shrubs
    cond1 = before.eq(12) \
        .And(after.eq(12)) \
        .And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    # water-green-water
    cond2 = before.eq(2).Or(before.eq(3)).Or(before.eq(4)) \
        .And(after.eq(2).Or(after.eq(3)).Or(after.eq(4))) \
        .And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    # tree-shrub-tree
    cond3 = before.eq(6) \
        .And(after.eq(6)) \
        .And(middle.eq(12))

    # crop-shrub-crop
    cond4 = before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11)) \
        .And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))) \
        .And(middle.eq(12))

    # crop-barren-crop
    cond5 = before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11)) \
        .And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))) \
        .And(middle.eq(7))

    # tree-farm-tree
    cond6 = before.eq(6) \
        .And(after.eq(6)) \
        .And(middle.eq(8).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    # farm-tree-farm
    cond7 = before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11)) \
        .And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))) \
        .And(middle.eq(6))

    # BU-tree-BU
    cond8 = before.eq(1) \
        .And(after.eq(1)) \
        .And(middle.eq(6))

    # tree-BU-tree
    cond9 = before.eq(6) \
        .And(after.eq(6)) \
        .And(middle.eq(1))

    # BU-farm-BU
    cond10 = before.eq(1) \
        .And(after.eq(1)) \
        .And(middle.eq(8).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    # barren-green-barren
    cond11 = before.eq(7) \
        .And(after.eq(7)) \
        .And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    zero_image = zero_image.add(cond1).add(cond2).add(cond3).add(cond4).add(cond5).add(cond6).add(cond7) \
        .add(cond8).add(cond9).add(cond10).add(cond11)

def process_conditions(before, middle, after, zero_image, th1, th2, i, length, L1_asset_new):

    cond1 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(12)) \
        .And(after.eq(12)) \
        .And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    cond2 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(2).Or(before.eq(3)).Or(before.eq(4))) \
        .And(after.eq(2).Or(after.eq(3)).Or(after.eq(4))) \
        .And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    cond3 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(6)) \
        .And(after.eq(6)) \
        .And(middle.eq(12))

    cond4 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11))) \
        .And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))) \
        .And(middle.eq(12))

    cond5 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11))) \
        .And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))) \
        .And(middle.eq(7))

    cond6 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(6)) \
        .And(after.eq(6)) \
        .And(middle.eq(8).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    cond7 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11))) \
        .And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))) \
        .And(middle.eq(6))

    cond8 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(1)) \
        .And(after.eq(1)) \
        .And(middle.eq(6))

    cond9 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(6)) \
        .And(after.eq(6)) \
        .And(middle.eq(1))

    cond10 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(1)) \
        .And(after.eq(1)) \
        .And(middle.eq(8).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    cond11 = zero_image.gte(th1).And(zero_image.lte(th2)) \
        .And(before.eq(7)) \
        .And(after.eq(7)) \
        .And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))


    if i != 2:
        middle = middle.where(cond1, 12)
        middle = middle.where(cond2, 7)
        middle = middle.where(cond3, 6)
        middle = middle.where(cond6, 6)

        cropping_frequency_img = crop_freq_array[i]
        middle = middle.where(cond7, cropping_frequency_img)
        middle = middle.where(cond4, cropping_frequency_img)
        middle = middle.where(cond5, cropping_frequency_img)

        middle = middle.where(cond9, 6)

        before = before.where(cond10, crop_freq_array[i-1])
        middle = middle.where(cond10, crop_freq_array[i])
        after = after.where(cond10, crop_freq_array[i+1])

        middle = middle.where(cond11, 7)

    if i != 1 and i != length-2 :
        cond8 = cond8.And(L1_asset_new[i-2].eq(1).And(L1_asset_new[i+2].eq(1)))
        middle = middle.where(cond8, 1)

    return {
        'before': before,
        'middle': middle,
        'after': after
    }

# applying condition for each pixel with count 1
for i in range(1, length-1):

    before = L1_asset_new[i - 1]
    middle = L1_asset_new[i]
    after = L1_asset_new[i + 1]

    updated_images = process_conditions(before, middle, after, zero_image, 1, 1, i, length, L1_asset_new)

    L1_asset_new[i - 1] = updated_images['before']
    L1_asset_new[i] = updated_images['middle']
    L1_asset_new[i + 1] = updated_images['after']

# applying condition for each pixel with count 2
## first iterate on 2019 windows and do the conditions
for i in range(1, length-2):

    before = L1_asset_new[i - 1]
    middle = L1_asset_new[i]
    after = L1_asset_new[i + 1]

    updated_images = process_conditions(before, middle, after, zero_image, 2, length-4, i, length, L1_asset_new)

    L1_asset_new[i - 1] = updated_images['before']
    L1_asset_new[i] = updated_images['middle']
    L1_asset_new[i + 1] = updated_images['after']

## iterate on whole again to do if any remaining conditions
for i in range(1, length-1):

    before = L1_asset_new[i - 1]
    middle = L1_asset_new[i]
    after = L1_asset_new[i + 1]

    updated_images = process_conditions(before, middle, after, zero_image, 2, length-4, i, length, L1_asset_new)

    L1_asset_new[i - 1] = updated_images['before']
    L1_asset_new[i] = updated_images['middle']
    L1_asset_new[i + 1] = updated_images['after']
    
    #temporal correction for first year
    first_year_image = L1_asset_new[0]

# BU-farm-farm
cond4 = L1_asset_new[0].eq(1) \
        .And(L1_asset_new[1].eq(8).Or(L1_asset_new[1].eq(9)).Or(L1_asset_new[1].eq(10)).Or(L1_asset_new[1].eq(11))) \
        .And(L1_asset_new[2].eq(8).Or(L1_asset_new[2].eq(9)).Or(L1_asset_new[2].eq(10)).Or(L1_asset_new[2].eq(11)))
# BU-tree-tree
cond5 = L1_asset_new[0].eq(1) \
        .And(L1_asset_new[1].eq(6)) \
        .And(L1_asset_new[2].eq(6))

first_year_image = first_year_image.where(cond5, 6)

cropping_frequency_img = crop_freq_array[0]
first_year_image = first_year_image.where(cond4, cropping_frequency_img)

L1_asset_new[0] = first_year_image



NameError: name 'crop_freq_array' is not defined

In [ ]:
for i in range(0, len(L1_asset_new)):
    image_export_task = ee.batch.Export.image.toAsset(
        image = L1_asset_new[i].clip(roi_boundary.geometry()),
        description = final_output_filename_array_new[i],
        assetId = final_output_assetid_array_new[i],
        pyramidingPolicy = {'predicted_label': 'mode'},
        scale = scale,
        maxPixels = 1e13,
        crs = 'EPSG:4326'
    )
    image_export_task.start()
